<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/node_postprocessor/ColbertRerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colbert Rerank

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


[Colbert](https://github.com/stanford-futuredata/ColBERT): ColBERT is a fast and accurate retrieval model, enabling scalable BERT-based search over large text collections in tens of milliseconds.

This example shows how we use Colbert-V2 model as a reranker.

In [2]:
!pip install llama-index
!pip install llama-index-core
!pip install --quiet transformers torch
!pip install llama-index-embeddings-openai
!pip install llama-index-llms-openai
!pip install llama-index-postprocessor-colbert-rerank

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.2 MB/s eta 0:00:00


In [49]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)

Download Data

In [50]:
import os

os.environ["OPENAI_API_KEY"] = "sk-"

In [51]:
# TO-DO: GET THE DOCUMENTS FROM ROUTING AGENT - all the answers in paragraph form in one document
!mkdir -p 'data/routing_agent/'
#!wget 'https://raw.githubusercontent.com/aishwarya-balaji/reranking/main/data/data8/data8.txt' -O 'data/data8/data8.txt'

In [52]:
# TO-DO: GET THE DOCUMENTS FROM ROUTING AGENT - all the answers in paragraph form in one document
with open('./data/routing_agent/routing_agent.txt', 'r') as file:
    # Read the entire file content into a string
    response_options = file.read()

In [53]:
documents = SimpleDirectoryReader("./data/routing_agent/").load_data()


In [54]:
from llama_index.core.node_parser import SentenceWindowNodeParser

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

nodes = node_parser.get_nodes_from_documents(documents)

In [55]:
from llama_index.core.node_parser import SentenceSplitter
text_splitter = SentenceSplitter()
base_nodes = text_splitter.get_nodes_from_documents(documents)

In [56]:
sentence_index = VectorStoreIndex(nodes)
base_index = VectorStoreIndex(base_nodes)

#### Retrieve top 10 most relevant nodes, then filter with Colbert Rerank

In [57]:
from llama_index.postprocessor.colbert_rerank import ColbertRerank

colbert_reranker = ColbertRerank(
    top_n=3,#will be equal to how many tools are executed
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

query_engine = sentence_index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[colbert_reranker],
)

In [58]:
#To Do: GET THE ORIGINAL QUERY FROM QD - get that from another file, so read the document
with open('original_query.txt', 'r') as file:
    # Read the entire file content into a string
    original_query = file.read()

response = query_engine.query(
    original_query,
)

In [59]:
print("Total nodes in response:", len(response.source_nodes))  # This will print how many nodes are there

Total nodes in response: 3


In [60]:
# TO DO: figure out how to acess scores for all nodes
for node in response.source_nodes:
    print(node.id_)
    print(node.node.get_content()[:150]) # number of characters - but not the paragraphs
    print("reranking score: ", node.score)
    print("retrieval score: ", node.node.metadata["retrieval_score"])
    print("**********")

b22c4ae8-2c0d-4255-b790-8e90ca60f3f0
Known as the "City of Light," Paris is celebrated for its romantic ambiance, iconic landmarks like the Eiffel Tower and Notre-Dame Cathedral, world-cl
reranking score:  0.7583699226379395
retrieval score:  0.8774996666312472
**********
f5b56c8c-3c02-4773-aaa0-6d658e22548d
The Eiffel Tower, one of the most famous landmarks in the world, stands as an enduring symbol of Paris. 
reranking score:  0.6390765309333801
retrieval score:  0.8489044372934368
**********
00170fbf-3c0d-448b-931d-5a5eb06008a9
Famous for its stunning sunsets, whitewashed buildings, blue-domed churches, and crystalline waters, Santorini is a dream destination for honeymooners
reranking score:  0.6205340623855591
retrieval score:  0.7898268505796289
**********


In [61]:
print(response)

Paris is known for its romantic ambiance, iconic landmarks like the Eiffel Tower and Notre-Dame Cathedral, world-class museums such as the Louvre and Musée d'Orsay, and charming neighborhoods like Montmartre and Le Marais.


In [62]:
response = str(response)

In [63]:
#Write response to another file

with open("reranking_result.txt", "w") as file:
    file.write(response)